In [1]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df=pd.read_csv('wiki_movie_plots_deduped_cleaned_genre_and_plot.csv')
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...
...,...,...,...,...,...,...,...,...,...,...
42530,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,NaN,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ...",film begin wxyz world war ended centre around ...
42531,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",two musician salih gürkan described adventure ...
42532,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",zafer sailor living mother döndü coastal villa...
42533,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...


In [3]:
data=pd.read_csv("most_frequent_words_200.csv")
data

,Genre,Words,Counts
0,western,town,1144
1,western,men,940
2,western,take,683
3,western,back,672
4,western,one,671
...,...,...,...
5595,anime,tima,35
5596,anime,leaving,34
5597,anime,age,34
5598,anime,destroyed,34


In [4]:
dict_words={i:(data[data['Genre']==i]['Words']).to_list() for i in data['Genre'].unique()}
dict_words


{'western': ['town',
  'men',
  'take',
  'back',
  'one',
  'kill',
  'man',
  'ranch',
  'horse',
  'find',
  'ride',
  'killed',
  'go',
  'gang',
  'return',
  'two',
  'get',
  'gun',
  'sheriff',
  'tell',
  'indian',
  'shoot',
  'father',
  'brother',
  'help',
  'try',
  'woman',
  'gold',
  'new',
  'come',
  'john',
  'gene',
  'train',
  'friend',
  'money',
  'son',
  'cattle',
  'shot',
  'make',
  'way',
  'also',
  'later',
  'escape',
  'time',
  'love',
  'away',
  'life',
  'outlaw',
  'old',
  'wife',
  'leave',
  'war',
  'see',
  'fight',
  'jim',
  'give',
  'order',
  'saloon',
  'dead',
  'wagon',
  'attack',
  'meet',
  'want',
  'billy',
  'daughter',
  'posse',
  'end',
  'however',
  'day',
  'tom',
  'home',
  'run',
  'arrives',
  'turn',
  'bill',
  'army',
  'land',
  'decides',
  'apache',
  'plan',
  'set',
  'leaf',
  'young',
  'death',
  'killing',
  'fort',
  'jack',
  'wxyz',
  'begin',
  'three',
  'night',
  'offer',
  'local',
  'year',
  'mar

In [5]:
def reduce_words(x):
    return ' '.join([i for i in x['Plot_cleanned'].split() if i in  dict_words[x['Genre_updated']]])
     

In [6]:
%%time
Train=df[~pd.isnull(df['Genre_updated'])]
Test=df[pd.isnull(df['Genre_updated'])]
Train['Plot_trimmed']=Train[['Genre_updated','Plot_cleanned']].apply(lambda x: reduce_words(x),axis=1)

Wall time: 1min 26s


<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize

In [8]:
Train

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned,Plot_trimmed
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...,film open two bandit breaking railroad telegra...,two bandit force train order train stop two ba...
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...,film family move suburb hoping quiet life thin...,film family move life thing start go wife get ...
10,1906,Dream of a Rarebit Fiend,American,Wallace McCutcheon and Edwin S. Porter,NaN,short,short,https://en.wikipedia.org/wiki/Dream_of_a_Rareb...,The Rarebit Fiend gorges on Welsh rarebit at a...,rarebit fiend gorge welsh rarebit restaurant l...,leaf begin get start try onto around man help ...
11,1906,From Leadville to Aspen: A Hold-Up in the Rockies,American,Francis J. Marion and Wallace McCutcheon,NaN,short action/crime western,action,https://en.wikipedia.org/wiki/From_Leadville_t...,The film features a train traveling through th...,film feature train traveling rockies hold crea...,film train two make along later
12,1906,From Leadville to Aspen: A Hold-Up in the Rockies,American,Francis J. Marion and Wallace McCutcheon,NaN,short action/crime western,western,https://en.wikipedia.org/wiki/From_Leadville_t...,The film features a train traveling through th...,film feature train traveling rockies hold crea...,train two make along later horse
...,...,...,...,...,...,...,...,...,...,...,...
42526,2013,Particle (film),Turkish,Erdem Tepegöz,"Jale Arıkan, Rüçhan Caliskur, Özay Fecht, Remz...",drama film,drama,https://en.wikipedia.org/wiki/Particle_(film),"Zeynep lost her job at weaving factory, and he...",zeynep lost job weaving factory family mother ...,job family mother daughter money live try find...
42531,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",two musician salih gürkan described adventure ...,two
42532,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",zafer sailor living mother döndü coastal villa...,mother girlfriend father also friend try help ...
42533,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...,film around young woman named however two go a...


In [9]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 5000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(Train['Plot_trimmed'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 1068 unique tokens.


In [10]:
X = tokenizer.texts_to_sequences(Train['Plot_trimmed'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (35692, 250)


In [11]:
Test

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...
...,...,...,...,...,...,...,...,...,...,...
42482,2009,Orada,Turkish,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Orada,When the matriarch of an Istanbul family dies ...,matriarch istanbul family dy home elderly son ...
42527,2014,Mandıra Filozofu,Turkish,Director: Müfit Can Saçıntı,Director: Müfit Can Saçıntı\r\nCast: Rasim Özt...,unknown,NaN,https://en.wikipedia.org/wiki/Mand%C4%B1ra_Fil...,Cavit an ambitious industralist in İstanbul pl...,cavit ambitious industralist i̇stanbul plan bu...
42528,2014,Winter Sleep,Turkish,Director: Nuri Bilge Ceylan,Director: Nuri Bilge Ceylan\r\nCast: Haluk Bil...,unknown,NaN,https://en.wikipedia.org/wiki/Winter_Sleep_(film),"Aydın, a former actor, owns a mountaintop hote...",aydın former actor owns mountaintop hotel capp...
42529,2014,Sivas,Turkish,Director: Kaan Müjdeci,Director: Kaan Müjdeci\r\nCast: Dogan Izci,unknown,NaN,https://en.wikipedia.org/wiki/Sivas_(film),The film follows an eleven-year-old boy named ...,film follows elevenyearold boy named aslan dog...


In [12]:
X_test1 = tokenizer.texts_to_sequences(Test['Plot_cleanned'].values)
X_test1 = pad_sequences(X_test1, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test1.shape)

Shape of data tensor: (6843, 250)


In [13]:
Y = pd.get_dummies(Train['Genre_updated']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (35692, 28)


In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(24984, 250) (24984, 28)
(10708, 250) (10708, 28)


In [15]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(28, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 100)          500000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 250, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 28)                2828      
Total params: 583,228
Trainable params: 583,228
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
352/352 [==============================] - 319s 906ms/step - loss: 2.1300 - accuracy: 0.3749 - val_loss: 1.7691 - val_accuracy: 0.4750
Epoch 2/5
352/352 [==============================] - 339s 964ms/step - loss: 1.5935 - accuracy: 0.5250 - val_loss: 1.1962 - val_accuracy: 0.6367
Epoch 3/5
352/352 [==============================] - 346s 982ms/step - loss: 1.0246 - accuracy: 0.6963 - val_loss: 0.8629 - val_accuracy: 0.7563
Epoch 4/5
352/352 [==============================] - 412s 1s/step - loss: 0.7926 - accuracy: 0.7669 - val_loss: 0.7291 - val_accuracy: 0.7959
Epoch 5/5
352/352 [==============================] - 342s 971ms/step - loss: 0.6655 - accuracy: 0.8021 - val_loss: 0.6305 - val_accuracy: 0.8211


In [17]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

335/335 [==============================] - 13s 38ms/step - loss: 0.6549 - accuracy: 0.8089
Test set
  Loss: 0.655
  Accuracy: 0.809


In [18]:
y_pred=model.predict(X_test)

In [19]:
model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([2, 1, 8, ..., 5, 5, 4], dtype=int64)

In [20]:
dx=pd.DataFrame(model.predict(X_test1),columns=['action', 'adventure', 'animated', 'anime', 'biography', 'comedy',
       'crime', 'documentary', 'drama', 'family', 'fantasy', 'historical',
       'horror', 'martial_arts', 'musical', 'mystery', 'noir', 'romance',
       'sci_fi', 'serial', 'short', 'social', 'sports', 'spy', 'teen',
       'thriller', 'war', 'western'])

In [21]:
dx

,action,adventure,animated,anime,biography,comedy,crime,documentary,drama,family,...,sci_fi,serial,short,social,sports,spy,teen,thriller,war,western
0,0.000703,0.006371,0.001864,0.006637,0.001168,1.067636e-05,0.002491,0.002023,0.000062,0.001314,...,0.006934,0.083195,0.010111,0.008358,0.000594,0.018622,0.001203,0.002227,0.021900,0.182411
1,0.000072,0.000048,0.005715,0.022781,0.006700,7.401490e-05,0.000527,0.007482,0.000079,0.013857,...,0.001464,0.003888,0.018321,0.698031,0.007815,0.002751,0.024889,0.001958,0.000413,0.000120
2,0.000059,0.007386,0.000552,0.009987,0.012941,3.354630e-06,0.000477,0.035403,0.000005,0.001274,...,0.000693,0.441444,0.038107,0.004940,0.009198,0.057522,0.001705,0.000057,0.241400,0.018145
3,0.000165,0.002350,0.000502,0.016584,0.009996,4.954701e-07,0.000899,0.024198,0.000004,0.001160,...,0.002584,0.298596,0.016529,0.025161,0.006524,0.090386,0.002024,0.000166,0.106368,0.013279
4,0.000221,0.010330,0.335183,0.066846,0.004414,3.816198e-05,0.000007,0.001025,0.000018,0.023446,...,0.088436,0.002012,0.169578,0.000984,0.001603,0.002805,0.002280,0.000173,0.000925,0.000097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6838,0.002492,0.433434,0.063573,0.049913,0.005778,1.403568e-04,0.000032,0.001806,0.000051,0.038808,...,0.142872,0.003471,0.006964,0.000314,0.001885,0.004526,0.003849,0.000452,0.005301,0.001278
6839,0.000296,0.000468,0.017990,0.017255,0.131788,3.672540e-04,0.000184,0.018179,0.000053,0.226908,...,0.002431,0.000974,0.021357,0.234942,0.077748,0.001331,0.124943,0.000390,0.000734,0.000121
6840,0.000240,0.000389,0.010707,0.013622,0.179751,7.398009e-04,0.000330,0.015629,0.000464,0.124659,...,0.002620,0.000746,0.014608,0.273450,0.040902,0.001073,0.140738,0.001739,0.000671,0.000115
6841,0.000101,0.016564,0.090425,0.033348,0.031649,4.271005e-05,0.000011,0.004180,0.000014,0.143926,...,0.018530,0.004858,0.127279,0.003070,0.007574,0.002062,0.011256,0.000107,0.002715,0.000299


In [22]:
def convert(x):
    if x>0.1:
        return 1
    return 0
    
for i in dx.columns:
    dx[i]=dx[i].apply(lambda x:convert(x))

In [23]:
dx.sum(axis=1).value_counts()

2    2189
3    2095
4    1298
1     991
5     243
6      14
0      13
dtype: int64

In [24]:
def genre_pred(col,x):

    if x==1:
        return col

for i in dx.columns:

    dx[i]=dx[i].apply(lambda x:genre_pred(i,x))

In [25]:
dx_list=dx.to_numpy().tolist()

In [26]:
Test['Genre_pred']=dx_list

<ipython-input-26-1e867a2d28ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test['Genre_pred']=dx_list


In [27]:
Test['Genre_pred']=Test['Genre_pred'].apply(lambda x:[i for i in x if i])

<ipython-input-27-5d7dce274ac8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test['Genre_pred']=Test['Genre_pred'].apply(lambda x:[i for i in x if i])


In [28]:
Test

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned,Genre_pred
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...,"[horror, mystery, western]"
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...,[social]
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...,"[serial, war]"
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...,"[mystery, serial, war]"
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...,"[animated, fantasy, short]"
...,...,...,...,...,...,...,...,...,...,...,...
42482,2009,Orada,Turkish,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Orada,When the matriarch of an Istanbul family dies ...,matriarch istanbul family dy home elderly son ...,"[adventure, fantasy, sci_fi]"
42527,2014,Mandıra Filozofu,Turkish,Director: Müfit Can Saçıntı,Director: Müfit Can Saçıntı\r\nCast: Rasim Özt...,unknown,NaN,https://en.wikipedia.org/wiki/Mand%C4%B1ra_Fil...,Cavit an ambitious industralist in İstanbul pl...,cavit ambitious industralist i̇stanbul plan bu...,"[biography, family, social, teen]"
42528,2014,Winter Sleep,Turkish,Director: Nuri Bilge Ceylan,Director: Nuri Bilge Ceylan\r\nCast: Haluk Bil...,unknown,NaN,https://en.wikipedia.org/wiki/Winter_Sleep_(film),"Aydın, a former actor, owns a mountaintop hote...",aydın former actor owns mountaintop hotel capp...,"[biography, family, social, teen]"
42529,2014,Sivas,Turkish,Director: Kaan Müjdeci,Director: Kaan Müjdeci\r\nCast: Dogan Izci,unknown,NaN,https://en.wikipedia.org/wiki/Sivas_(film),The film follows an eleven-year-old boy named ...,film follows elevenyearold boy named aslan dog...,"[family, fantasy, short]"


In [29]:
Test.to_csv('Test.csv',index=False)